## Лекция 4 - Элементы функционального программирования в Python

- синтаксис функций
- LEGB
- лямбды
- filter, map, zip
- генераторы и итераторы
- декораторы
- модуль functools
- модуль itertools (is back)

In [1]:
# пример простейших функций
# (уже не ново, т.к. видели примеры в пакете algo)

def foo(x):
    return x + 10

def bar():
    print('Hello')
    
magic_number = foo(63)
print(magic_number)

bar()

73
Hello


In [2]:
# функция может возвращать все что угодно:
# удобно, т.к. можно вернуть несколько значений сразу
# (по сути кортеж)
def arithmetics(a, b):
    ''' функция возвращает сумму, разность и произведение чисел '''
    summa = a + b
    diff = a - b
    mult = a * b
    return summa, diff, mult

s = arithmetics(10, 15)
s

(25, -5, 150)

In [3]:
_, difference, _ = arithmetics(7, 9)
difference

-2

In [4]:
# Синтаксис *args, **kwargs:
def pretty_print(text, *args, **kwargs):
    for cnt in args:
        print(kwargs['fill'] * cnt)
        
    print(text)
    
    if kwargs['footer']:
        for cnt in reversed(args):
            print(kwargs['fill'] * cnt)
            
    print()        
    print('args:')
    print(args)
    print('kwargs:')
    print(kwargs)
    
pretty_print('Hello', 20, 10, 5, fill='~', footer=True)

~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~
~~~~~
Hello
~~~~~
~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~

args:
(20, 10, 5)
kwargs:
{'footer': True, 'fill': '~'}


In [5]:
param_list = [50, 25, 10]
pretty_print('Hello!', param_list, fill='~', footer=False)

TypeError: can't multiply sequence by non-int of type 'list'

In [8]:
pretty_print('Hello!', *param_list, fill='~', footer=False)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~
Hello!

args:
(50, 25, 10)
kwargs:
{'footer': False, 'fill': '~'}


In [9]:
# очень важно: функция - это first-class объект!
dir(pretty_print)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [10]:
# немного поинстроспектируем:
print('Qualified name:', pretty_print.__qualname__)
print('Vars:', pretty_print.__code__.co_varnames)

Qualified name: pretty_print
Vars: ('text', 'args', 'kwargs', 'cnt')


In [11]:
# эквивалентно foo(10)
foo.__call__(10)

20

In [12]:
# таким образом, функции можно присваивать переменным:
cooler_foo = foo
cooler_foo(100)

110

In [13]:
# можно делать массив из функций
# и последовательно выполнять их:
import math

funcs = [foo, math.factorial]

for func in funcs:
    print(func.__qualname__, func(5))

foo 15
factorial 120


In [14]:
# можно передавать функцию в функцию:
def process_elementwise(seq, func):
    new_seq = [func(s) for s in seq]
    return new_seq

words = ['Fundamentals', 'of', 'brainwashing...']
process_elementwise(words, str.__len__)

[12, 2, 15]

In [15]:
process_elementwise(words, str.upper)

['FUNDAMENTALS', 'OF', 'BRAINWASHING...']

In [16]:
nums = [1, -3, 5, -7, -9]
process_elementwise(nums, abs)

[1, 3, 5, 7, 9]

In [17]:
process_elementwise(nums, foo)

[11, 7, 15, 3, 1]

### Итераторы и генераторы:
- итератор - это концепция (любой объект, имеющий методы ```next()``` и ```__iter__()```)
- генератор - языковое средство (объект вокруг функции с ```yield```)

Любой генератор является итератором, но не наоборот.

In [18]:
def odd_iterator(x):
    for i in range(1, x, 2):
        yield i

for x in odd_iterator(10):
    print(x)

1
3
5
7
9


In [19]:
it = odd_iterator(10)
it

<generator object odd_iterator at 0x02D6BBA0>

In [20]:
print(next(it))
print(next(it))

1
3


In [21]:
try:
    it.throw(ValueError)
except:
    print('Caught')    

it.close()

Caught


In [22]:
next(it)

StopIteration: 

In [24]:
# еще пример генератора (генерирующего цифры числа)
def generate_digits(x):
    while x > 0:
        yield x % 10
        x = x // 10

for digit in generate_digits(5716):
    print(digit)

6
1
7
5


In [25]:
l = [x for x in generate_digits(54321)]
l

[1, 2, 3, 4, 5]

In [26]:
i = generate_digits(4523)
print([next(i), next(i), next(i)])

[3, 2, 5]


In [27]:
# еще пример генератора (генерирующего перестановки)
import itertools

def permute(n):
    for perm in itertools.permutations(range(1, n+1)):
        yield perm
        
for p in permute(3):
    print(p)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


In [28]:
# еще раз о разнице между range в python2 и python3
nums = range(1, 5)
nums

# в Python2 будет выведено [1, 2, 3, 4]
# аналог: nums = xrange(1, 5)

range(1, 5)

In [29]:
# range - специфический итерируемый объект
# с дополнительными возможностями (например, индексирование)
dir(nums)

['__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index',
 'start',
 'step',
 'stop']

In [30]:
# Посчитаем сумму цифр числа в декларативном стиле:
n = 123
print(sum(map(int, str(n))))

6


In [31]:
from functools import reduce

s = range(1, 5)
reduce(lambda x, y: x + y, s)

10

In [32]:
# Декораторы

def decorate(f):
    def new(*args, **kwargs):
        print(" ======= BEGIN ")
        f(args[0])
        print(" ======= END ")

    # декоратор возвращает функцию (callable)
    return new

# декорируем функцию
@decorate
def hello(x):
    print(x)

hello("OK")

# происходит такой вызов:
# decorate(hello("OK"))()

 ======= BEGIN 
OK
 ======= END 
